In [1]:
from pyomo.environ import *
from torchswarm.functions import Function
import numpy as np
import os
import sys
import pandas as pd
import pandapower as pp
import torch
from matplotlib import pyplot as plt
import pandapower.networks as pn
import time
from scipy import sparse
from copy import deepcopy
sys.path.append('..')
import src.data_processing.power_parser as parser
from src.model.acopf_pyomo import spawn_ac_opf_model, get_params_for_model
from src.model.cno import *
from torchswarm.utils.parameters import SwarmParameters
import warnings
warnings.filterwarnings('ignore')
import logging

logging.getLogger('pyomo.core').setLevel(logging.ERROR)

#PYOMO HAS DIFFEQ???????????

In [2]:
net = pn.case30()
net["load"]['p_mw'] = net["load"]['p_mw'] * 0.9
net["load"]['q_mvar'] = net["load"]['q_mvar']
net["gen"]['max_p_mw'] = net["gen"]['max_p_mw'] * 2
net["gen"]['max_q_mvar'] = net["gen"]['max_q_mvar'] * 2
net["ext_grid"]['max_p_mw'] = net["ext_grid"]['max_p_mw'] 
net["ext_grid"]['max_q_mvar'] = net["ext_grid"]['max_q_mvar']
net["poly_cost"]["cp2_eur_per_mw2"] = net["poly_cost"]["cp2_eur_per_mw2"] * 0
net, bs, S, G, B, n_gens, n_buses, n_branches, gen_idxs, bus_idxs, branch_idxs = get_params_for_model(net)
S = 2 * S

In [3]:
def spawn_foo():
    return spawn_ac_opf_model(net, bs, S, G, B, n_gens, n_buses, n_branches, gen_idxs, bus_idxs, branch_idxs)
sample_model = spawn_foo()
solver = SolverFactory('mindtpy')
nvars = sum(1 for v in sample_model.component_data_objects(Var))
criterion = ObjectiveCNO(sample_model)
empso = SwarmOptimizerCNOPyomo(nvars, 5, swarm_optimizer_type="exponentially_weighted", max_iterations=10, init_model=spawn_foo, solver=solver)
# for i in range(len(empso.swarm)):
#     empso.swarm[i].position = x0 + torch.randn_like(x0) * 1e-2
empso.optimize(criterion)

    functionality.  (deprecated in 6.4.2) (called from
    /Users/aleks/opt/anaconda3/envs/cno_acopf/lib/python3.10/site-
    packages/pyomo/contrib/mindtpy/initialization.py:155)


In [4]:
print(empso.run(verbosity=True).__dict__)

Iteration 1 >> global best fitness 229.817  | iteration time 5.447
Iteration 2 >> global best fitness 229.515  | iteration time 4.393
Iteration 3 >> global best fitness 229.515  | iteration time 4.510
Iteration 4 >> global best fitness 229.515  | iteration time 4.228
Iteration 5 >> global best fitness 229.515  | iteration time 4.137
Iteration 6 >> global best fitness 229.515  | iteration time 4.238
Iteration 7 >> global best fitness 229.515  | iteration time 4.288
Iteration 8 >> global best fitness 229.515  | iteration time 4.115
Iteration 9 >> global best fitness 229.515  | iteration time 4.100
Iteration 10 >> global best fitness 229.515  | iteration time 4.114
{'r1': 0.4534424841403961, 'r2': 0.4304801821708679, 'gbest_position': tensor([ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00,
         1.0000e+00,  1.0353e+00,  1.0373e+00,  1.0238e+00,  1.0217e+00,
         1.0231e+00,  1.0168e+00,  1.0107e+00,  1.0047e+00,  1.0280e+00,
         1.0341e+00,  1.0280e+00,  1.03

In [5]:
pp.runopp(net)

cost_pp = net['res_cost']
gen_pp = np.array(list(net['res_gen'].p_mw) + list(net['res_ext_grid'].p_mw))
print(gen_pp)
print(cost_pp)

[9.85994668e+01 4.98910660e+01 2.51191370e+01 6.47377231e-06
 6.96901347e-06 2.71118856e-05]
304.0774227402238


In [6]:
# display(value(empso.gbest_particle.model.UC))
def extract_values(name):    
    for k in empso.gbest_particle.model.component_objects(Var):
        if k.name == name:
            output = k.get_values()
    return output
UC_vals=extract_values('UC')            
Pg_vals=extract_values('Pg')  
obj_pyomo = empso.gbest_particle.model.obj()

In [7]:
print("CNO")
print(UC_vals)
print(Pg_vals)
print(obj_pyomo)

CNO
{1: 1.0, 21: 1.0, 26: 1.0, 22: 1.0, 12: 1.0, 0: 1.0}
{1: 74.00872358571962, 21: 100.00000000636875, 26: -8.24916085034676e-09, 22: -7.9084920854786e-09, 12: -7.902826501376846e-09, 0: 3.3757139504075293e-10}
229.5152662078095


In [8]:
print("PP")
print(gen_pp)
print(cost_pp)

PP
[9.85994668e+01 4.98910660e+01 2.51191370e+01 6.47377231e-06
 6.96901347e-06 2.71118856e-05]
304.0774227402238
